In [34]:
train = 'train.csv'
test = 'test.csv'


In [35]:
import pandas as pd

df = pd.read_csv(train, index_col='PassengerId')
df_test = pd.read_csv(test, index_col='PassengerId')


In [26]:
drop_columns = ['Ticket']
passthrough_columns=['Pclass', 'SibSp', 'Parch']
target_column = ['Survived']

text_columns_manquant = ['Cabin']  
text_columns_sans_manquant = ['Name']

numeric_columns_manquant =['Age', 'Fare']
cat_manquant = ['Embarked']
cat_sans_manquant = ['Sex']


In [27]:
X = df.drop('Survived', axis=1)
y = df[['Survived']]


In [32]:
print(X.head())

print

             Pclass                                               Name  \
PassengerId                                                              
1                 3                            Braund, Mr. Owen Harris   
2                 1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
3                 3                             Heikkinen, Miss. Laina   
4                 1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
5                 3                           Allen, Mr. William Henry   

                Sex   Age  SibSp  Parch            Ticket     Fare Cabin  \
PassengerId                                                                
1              male  22.0      1      0         A/5 21171   7.2500   NaN   
2            female  38.0      1      0          PC 17599  71.2833   C85   
3            female  26.0      0      0  STON/O2. 3101282   7.9250   NaN   
4            female  35.0      1      0            113803  53.1000  C123   
5              male  35.0

In [28]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer 
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import FunctionTransformer




#pipeline pour le preprocesing des colonnes catégorielles avec des valeurs manquantes
cat_manquant_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder(handle_unknown='ignore'))


from fonction_custom import extract_cabin_letter #import de la fonction custom extract_cabin_letter

#pipeline pour le preprocesing de cabin

preprocessor_cabin = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    FunctionTransformer(extract_cabin_letter),
    OneHotEncoder(handle_unknown='ignore')
)

#make_column_transformer: permet de faire un preprocessing spécifique pour chaque colonne
preprocessor = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore'), cat_sans_manquant),
    (cat_manquant_pipeline, cat_manquant),
    (SimpleImputer(strategy='median'), numeric_columns_manquant),
    (CountVectorizer(), 'Name'),
    (preprocessor_cabin, 'Cabin'),
    ('passthrough', passthrough_columns),
    ('drop', drop_columns),
)



In [22]:
# pipeline final

from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV

In [33]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(bootstrap=True, criterion="entropy", max_features=0.8, min_samples_leaf=5, min_samples_split=12, n_estimators=100)

rf_pipeline = make_pipeline(preprocessor, rf)

rf_pipeline.fit(X, y)












ValueError: Expected a 2-dimensional container but got <class 'pandas.core.series.Series'> instead. Pass a DataFrame containing a single row (i.e. single sample) or a single column (i.e. single feature) instead.

In [29]:
#export du modele pour la production

import joblib

joblib.dump(rf_pipeline, 'titanic.model.pkl')

['titanic.model.pkl']

In [ ]:
#notez toute les versions des librairies utilisées

import joblib
print(joblib.__version__)

import sklearn
print(sklearn.__version__)

import pandas as pd
print(pd.__version__)

import numpy
print(numpy.__version__)




1.4.2
1.5.1
2.2.2
2.0.0
